In [1]:
!pip install autogen "httpx==0.27.2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.3/497.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.2/314.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.2 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [3]:
import autogen
from autogen import AssistantAgent, UserProxyAgent
from autogen.coding import LocalCommandLineCodeExecutor # 컴퓨터에서 명령어를 실행하고 그 결과를 가져오는 클래스
import os
os.environ["OPENAI_API_KEY"] = "sk" #openai 키 입력

config_list = [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]
llm_config = {
    "cache_seed": 42, # 결과를 항상 일정하게 만들어주는 값
    "temperature": 0, # 답변의 다양성을 제공하는 변
    "config_list": config_list, # config_list를 사용하여 API 키와 모델 정보를 전달
    "timeout": 160, # 요청이 처리될 최대 시간(초)
}

In [13]:
# 운동화 신제품 광고를 위해 공통, 제품 디자인팀, 마케팅 팀의 의견 제시
# 공통/일반 작업에 대한 프롬프트
task = '''
 **Task**: 운동화 신제품 광고를 위해 다음 요구 사항에 맞는 솔루션을 제시해야 합니다.
-연령, 성별, 소득 수준, 라이프스타일, 취미 등 세분화.
-고객의 운동화 사용 목적(스포츠, 패션, 일상 등) 분석.
-운동화 제품군 내 다른 제품들과의 차별화 전략.
-경쟁사 신제품 및 광고 전략 분석.
-현재 시장 트렌드와 소비자 선호도 파악.
-광고 제작, 촬영, 유통 채널별 비용 배분.
-인적 자원 및 외부 에이전시 활용 계획.
-신제품 출시 일정과 광고 캠페인 시점의 조율.
'''

# 제품 디자인팀에서 고려해야할 내용에 대한 프롬프트
design_prompt = '''
**Role**: 당신은 제품 디자인팀의 팀원입니다.
다음의 요구 사항을 기반으로 제품 디자인팀의 의견을 제시합니다:

-제품의 주요 특징 및 USP(Unique Selling Point) 도출
-제품의 세부 디테일과 장점을 강조할 수 있는 촬영 준비.
-초기 광고 시안에서 제기된 소비자 테스트 피드백 반영.
-운동화의 다양한 코디 및 용도(운동, 캐주얼, 정장 등)를 표 형태로 제시.
'''
design_prompt += task

# 마케팅 팀에서 고려해야할 내용에 대한 프롬프트
marketing_prompt = '''
**Role**: 당신은 마케팅 팀의 팀원입니다.
온라인 & 오프라인 마케팅을 고려하지 않고 다양한 마케팅 행사를 제안해야 합니다.

-광고 메시지 개발: 슬로건 및 핵심 메시지 작성.
-온라인 & 오프라인 채널별 전략을 비교하여 표 형태로 정리
-광고 도달률, 클릭률, 전환율 등 성과 측정.
-초기 프로모션(할인, 번들 판매 등) 준비.
-광고 후 고객 피드백 및 리뷰 관리.
'''
marketing_prompt += task

# PM이 고려해야할 내용에 대한 프롬프트
pm_prompt =  '''
**Role**: 당신은 디자인팀과 마케팅 팀의 의견을 수렴하는 PM입니다.

-당신은 제품 디자인팀과 마케팅팀이 제시한 의견의 단점을 지적하거나 추가 작업을 요청해야 합니다.
-각 의견을 검토한 후, 비즈니스 요구 사항에 따라 최상의 광고 전략을 도출해야 합니다.
-결정을 위해 필요한 만큼의 요약 표를 사용할 수 있습니다.
'''
pm_prompt += task

In [14]:
user_proxy = UserProxyAgent(
    name="proxy",
    system_message = "Hub for User and AssistantAgent", # 사용자와 AssistantAgent간의 허브 역할
    code_execution_config={
        "last_n_messages": 2, # 최근 2개의 메시지만 기억
        "work_dir": "multiagent", # 그룹 채팅 작업 디렉터리를 설정
    },
    human_input_mode="NEVER", # 사용자 입력을 받지 않고 자동으로 작업을 처리
) # 다른 에이전트들과 협력하여 메시지를 전달하거나 요청을 처리

design_agent = AssistantAgent(
    name = "design",
    system_message = design_prompt, # 디자인팀과 관련된 초기 프롬프트를 제공하여 에이전트의 역할을 지정
    llm_config={"config_list": config_list} # config_list을 통해 LLM 호출
    ) # 디자인 관련 요청을 처리하는 에이전트 (제품 디자인팀)

marketing_agent = AssistantAgent(
    name = "marketing",
    system_message = marketing_prompt, # 마케팅팀과 관련된 초기 프롬프트를 제공하여 에이전트의 역할을 지정
    llm_config={"config_list": config_list}
    ) # 마케팅 관련 작업을 처리하는 에이전트 (마케팅팀)

pm_agent = AssistantAgent(
    name = "pm",
    system_message = pm_prompt, # pm 역할을 수행하도록 초기 프롬프트를 제공
    llm_config={"config_list": config_list}
) # 홍보 pm으로서 다른 에이전트의 작업을 관리하고 결과를 조정 (PM)

In [15]:
def state_transition(latest_speaker, groupchat): # latest_speaker: 대화에서 마지막으로 발언한 에이전트(현재 상태), groupchat: 대화의 전체 컨텍스트를 담고 있는 객체
   messages = groupchat.messages

   if latest_speaker is user_proxy:
       return design_agent # 마지막 화자가 user_proxy라면 다음 상태로 design_agent를 반환
   elif latest_speaker is design_agent:
       return marketing_agent # 마지막 화자가 design_agent라면 다음 상태로 marketing_agent를 반환
   elif latest_speaker is marketing_agent:
       return pm_agent # 마지막 화자가 marketing_agent라면 다음 상태로 pm_agent를 반환
   elif latest_speaker is pm_agent:
       return None # 마지막 화자가 pm_agent라면 더 이상 상태 전환이 없으며, None을 반환하여 대화가 종료

In [16]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, design_agent, marketing_agent, pm_agent], # 대화에 참여하는 에이전트들의 리스트
    messages=[], # 초기 메시지의 리스트로, 비어 있는 상태([])로 시작
    max_round=4, # 대화의 최대 라운드 수 (무한 루프를 방지)
    speaker_selection_method=state_transition, # 대화에서 다음 화자를 결정하는 함수
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config) # GroupChatManager: 그룹 채팅의 전체 흐름을 관리하는 객체

user_proxy.initiate_chat(
    manager, message="이 비즈니스 요구 사항을 기반으로 최상의 운동화 광고 전략을 제안하세요."
) # 대화 시작 (user_proxy가 초기 메시지를 보냄으로써 대화가 시작)

proxy (to chat_manager):

이 비즈니스 요구 사항을 기반으로 최상의 운동화 광고 전략을 제안하세요.

--------------------------------------------------------------------------------

Next speaker: design

design (to chat_manager):

운동화 신제품 광고를 위한 전략은 다음과 같이 제안합니다.

1. **타겟 소비자 세분화**:
   - **연령**: 18-35세, 패션에 민감하고 트렌드에 민감한 젊은 층.
   - **성별**: 남녀 모두를 타겟으로 하여 다양한 컬러 옵션 제공.
   - **소득 수준**: 중상층, 패션이나 스포츠에 투자하는 것을 즐기는 소비자.
   - **라이프스타일**: 활동적이며 건강을 중시하고 개성을 표현하고자 하는 사람들.
   - **취미**: 런닝, 피트니스, 여행 등 활동적인 취미를 가진 사람들.

2. **고객의 운동화 사용 목적 분석**:
   - **스포츠**: 고성능 기능성을 제공하는 혁신적인 소재 사용.
   - **패션**: 최신 트렌드를 반영한 디자인과 다양한 컬러 옵션으로 스타일을 강조.
   - **일상**: 편안한 착용감과 내구성을 통한 일상적 사용 최적화.

3. **제품 차별화 전략**:
   - 독창적인 디자인 요소와 최신 기술(예: 통풍 소재, 경량 쿠션 등) 강조.
   - 맞춤형 피팅 옵션 제공.

4. **경쟁사 분석**:
   - 경쟁사의 최신 신제품과 광고 전략을 분석하고, 차별화된 USP(Unique Selling Point)를 강조.
   - 자사만의 독창적 브랜드 스토리를 기반으로 한 감성적 광고 캠페인 추진.

5. **시장 트렌드와 소비자 선호도**:
   - 지속 가능성 트렌드를 반영하여 친환경 소재 사용 강조.
   - 커스터마이징 옵션을 제공하여 소비자 참여 유도.

6. **광고 제작, 촬영, 유통 채널별 비용 배분**:
   - **광고 제작**: 전문 광

ChatResult(chat_id=None, chat_history=[{'content': '이 비즈니스 요구 사항을 기반으로 최상의 운동화 광고 전략을 제안하세요.', 'role': 'assistant', 'name': 'proxy'}, {'content': '운동화 신제품 광고를 위한 전략은 다음과 같이 제안합니다.\n\n1. **타겟 소비자 세분화**:\n   - **연령**: 18-35세, 패션에 민감하고 트렌드에 민감한 젊은 층.\n   - **성별**: 남녀 모두를 타겟으로 하여 다양한 컬러 옵션 제공.\n   - **소득 수준**: 중상층, 패션이나 스포츠에 투자하는 것을 즐기는 소비자.\n   - **라이프스타일**: 활동적이며 건강을 중시하고 개성을 표현하고자 하는 사람들.\n   - **취미**: 런닝, 피트니스, 여행 등 활동적인 취미를 가진 사람들.\n\n2. **고객의 운동화 사용 목적 분석**:\n   - **스포츠**: 고성능 기능성을 제공하는 혁신적인 소재 사용.\n   - **패션**: 최신 트렌드를 반영한 디자인과 다양한 컬러 옵션으로 스타일을 강조.\n   - **일상**: 편안한 착용감과 내구성을 통한 일상적 사용 최적화.\n\n3. **제품 차별화 전략**:\n   - 독창적인 디자인 요소와 최신 기술(예: 통풍 소재, 경량 쿠션 등) 강조.\n   - 맞춤형 피팅 옵션 제공.\n\n4. **경쟁사 분석**:\n   - 경쟁사의 최신 신제품과 광고 전략을 분석하고, 차별화된 USP(Unique Selling Point)를 강조.\n   - 자사만의 독창적 브랜드 스토리를 기반으로 한 감성적 광고 캠페인 추진.\n\n5. **시장 트렌드와 소비자 선호도**:\n   - 지속 가능성 트렌드를 반영하여 친환경 소재 사용 강조.\n   - 커스터마이징 옵션을 제공하여 소비자 참여 유도.\n\n6. **광고 제작, 촬영, 유통 채널별 비용 배분**:\n   - **광고 제작**: 전문 광고 제작사와 협업하여 고퀄리티의 시각적 콘텐츠